In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install convokit

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd

In [4]:
from convokit import Corpus, download

corpus = Corpus(filename=download("subreddit-zelda"))

Dataset already exists at /home/taylor/.convokit/downloads/subreddit-zelda


In [5]:
df = corpus.get_utterances_dataframe()
df = df[df['speaker'] != '[deleted]']
df

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,vectors
id,,,,,,,,,,,,,,,
cktif,1277961530,,mussedeq,None,cktif,19,None,1522932038,0,None,zelda,False,/r/zelda/comments/cktif/b_on_zelda/,,[]
clwk9,1278262867,,rich4rd,None,clwk9,4,None,1522932831,0,None,zelda,False,/r/zelda/comments/clwk9/which_is_your_favourit...,,[]
cm9xu,1278369752,,DekuNut,None,cm9xu,4,None,1522933106,0,None,zelda,False,/r/zelda/comments/cm9xu/zelda_fans_are_never_h...,,[]
cn9nu,1278593645,,andreal,None,cn9nu,15,None,1522933839,0,None,zelda,False,/r/zelda/comments/cn9nu/zelda_timeline/,,[]
cpwis,1279205672,,andreal,None,cpwis,1,None,1522935763,0,None,zelda,False,/r/zelda/comments/cpwis/reality_of_links_inven...,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e8tk02q,1541029896,EEEYYYYAAAEUGHHHHH!,Vincep543,9syksc,9syksc,1,e8tk02q,1541688198,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",zelda,False,/r/zelda/comments/9syksc/i_created_some_zelda_...,,[]
e8tk3xf,1541030003,This looks amazing ! Almost too cool for skull.,ghjfjk,9t48hf,9t48hf,4,e8tk3xf,1541688245,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",zelda,False,/r/zelda/comments/9t48hf/my_younger_brother_as...,,[]
e8tkf6z,1541030317,Breath Of The Wild West,SonOfErdrick,9swcmu,9swcmu,3,e8tkf6z,1541688385,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",zelda,False,/r/zelda/comments/9swcmu/so_excited_to_finally...,,[]


In [6]:
# https://storage.googleapis.com/naacl21_account_linking/raw_mud.tar.gz

In [7]:
# Do something iteration based
# Grab by timerange, cluster get sim (1.5 std), Take rnadoms from sus groups and compare pre post tor (with random usernames)
# See when someone leaves the conversation and if can detect groups that way or when a new person enters the conversation
# look at privacy scopes og paper and think about what should be possible given different scopes and what 
# iterations would make sense given what scopes and features

In [8]:
# Grab a random convo of a certain min and max length and number of people talking
# Change convo into 2d array of when each person should talk (action list with time stamps)
conv_max_length = 50000
conv_min_length = 25
conv_min_users = 5
conv_max_users = 200
conv_length_mask = df.groupby("conversation_id")["conversation_id"].transform(len)
conv_users_mask = df.groupby("conversation_id")["speaker"].transform('nunique')

mask = (conv_length_mask >= conv_min_length) & (conv_length_mask <= conv_max_length) \
     & (conv_users_mask >= conv_min_users) & (conv_users_mask <= conv_max_users)

filtered_df = df[mask]

In [9]:
import numpy as np

convs = filtered_df["conversation_id"].unique()
print(len(convs))
random_conv = np.random.choice((convs))
chosen = filtered_df[filtered_df["conversation_id"] == random_conv]


11060


In [26]:
import os

def save_chat(convo_num, chat_log):
    num = -1
    for user in chat_log:
        num += 1
        filename = str(convo_num) + '/user_' + str(num) + '.csv'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'w') as f: 
            for msg in chat_log[user]:
                f.write(str(msg) + "\n")

In [27]:
num_good = 0
thres = 5
convs = filtered_df["conversation_id"].unique()
convo_num = 0
print(len(convs))
for random_conv in convs:

    #random_conv = np.random.choice((convs))
    chosen = filtered_df[filtered_df["conversation_id"] == random_conv]
    users = convs = chosen["speaker"].unique()
    start_time = chosen["timestamp"].min()
    chat_log = {}
    num_msgs = {}
    num_good = 0
    for user in users:
        num_msgs[user] = 0
        log = []
        for index, msg in chosen[chosen["speaker"] == user].iterrows():
            #print(msg)
            log += [msg['timestamp'] - start_time]
            num_msgs[user] += 1
        chat_log[user] = log
        if num_msgs[user] > 5:
                num_good += 1
    if num_good > thres:
        print(str(num_good) + ":t " + str(thres))
        save_chat(convo_num, chat_log)
        print('saved' + str(convo_num))
        convo_num += 1
num_good

11060
14:t 5
saved0
9:t 5
saved1
7:t 5
saved2
6:t 5
saved3
7:t 5
saved4
6:t 5
saved5
11:t 5
saved6
7:t 5
saved7
7:t 5
saved8
12:t 5
saved9
10:t 5
saved10
17:t 5
saved11
15:t 5
saved12
8:t 5
saved13
17:t 5
saved14
6:t 5
saved15
12:t 5
saved16
23:t 5
saved17
6:t 5
saved18
16:t 5
saved19
7:t 5
saved20
14:t 5
saved21
9:t 5
saved22
16:t 5
saved23
7:t 5
saved24
8:t 5
saved25
20:t 5
saved26
6:t 5
saved27
7:t 5
saved28
10:t 5
saved29
20:t 5
saved30
8:t 5
saved31
19:t 5
saved32
6:t 5
saved33
7:t 5
saved34
9:t 5
saved35
9:t 5
saved36
15:t 5
saved37
6:t 5
saved38
6:t 5
saved39
12:t 5
saved40
15:t 5
saved41
6:t 5
saved42
12:t 5
saved43
9:t 5
saved44
6:t 5
saved45
10:t 5
saved46
6:t 5
saved47
7:t 5
saved48
9:t 5
saved49
8:t 5
saved50
7:t 5
saved51
6:t 5
saved52
6:t 5
saved53
7:t 5
saved54
6:t 5
saved55
9:t 5
saved56
6:t 5
saved57
7:t 5
saved58
6:t 5
saved59
6:t 5
saved60
15:t 5
saved61
16:t 5
saved62
7:t 5
saved63
7:t 5
saved64
7:t 5
saved65
7:t 5
saved66
7:t 5
saved67
7:t 5
saved68
6:t 5
saved69
8

0

In [25]:
chat_log

{'appledas21': [0],
 'longjohndevine': [154],
 'BaBiTa87': [1290],
 'Monic_maker': [3142],
 'texasjoe': [3621],
 'ichuckle': [3672],
 'MadamMurloc': [5333],
 'gaylordmcqueergay': [5408],
 'gilwen0017': [5420, 56976],
 'TearsOfTheDragon': [5761],
 'IEatKaleAndBabies': [6321, 20081, 20173, 27109],
 'HansTheAxolotl': [7333, 7349, 8222],
 'ZeldaFierceDeity': [8988],
 'jackofallcards': [11033],
 'FatBoyLilPeeper': [12152],
 'el-toro-loco': [13715],
 'Cartoons4adults': [14651],
 'CajunTurkey': [15065, 26671, 27492],
 'furbit73': [15476, 15531],
 'Funkit': [15649],
 'Melancholia': [15785],
 'Davidson30': [16131, 23294],
 'justcuri0us': [17227],
 'logannev': [17537],
 'like-a-shark': [19382],
 'Shadow_Riptor': [20491],
 'Galaxy_Horror_Queen': [20635],
 'link87': [21490],
 'Darkiceflame': [22102],
 'watergo': [23828],
 'Master11205': [24276],
 'barsoomwarrior': [24438],
 'nonsensebearer': [24653],
 'supakame': [26117],
 'Xiefux': [27497],
 'RigasTelRuun': [27782],
 'Mhaerdel': [27797, 28199],
 

In [ ]:
# Compute overall stats(# people, avg response time, msgs per person, lifespan of convo, 
# avg message size, message size per person) for conversations per subreddit
# compare for top 25 subreddits and 25 random subreddits
# Viz results

In [ ]:
subreddit = "subreddit-Cornell"
corpus = Corpus(filename=download(subreddit))
df = corpus.get_utterances_dataframe()
df = df[df['speaker'] != '[deleted]']
df

In [ ]:
convo_groups = df.groupby("conversation_id")

In [ ]:
# # of people
num_users =convo_groups["speaker"].transform('nunique')
num_users.describe()

In [ ]:
# lifespan of convo
start = convo_groups["timestamp"].transform('min')
end = convo_groups["timestamp"].transform('max')
lifespan = end - start
lifespan.describe()

In [ ]:
# avg response time
(lifespan / num_users).describe()

In [ ]:
# avg message size
convo_groups["text"].apply(lambda x: np.mean(x.str.len())).describe()

In [ ]:
convo_groups_person = df.groupby(["conversation_id", "speaker"])

In [ ]:
# msgs per person
convo_groups_person["speaker"].transform(len).describe()

In [ ]:
# message size per person
convo_groups_person["text"].apply(lambda x: np.mean(x.str.len())).describe()